# This is a baseline Model that assigns random labels to the dev dataset 

In [9]:
import numpy as np
import math
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from copy import deepcopy
import pygeohash as pgh
%matplotlib inline



In [20]:
def geoencode(x):
    return pgh.encode(x['Latitude'], x['Longitude'], precision=7)

train_data = pd.read_csv("train.csv")
# Remove 67 outlier data points that have wrong Latitude and Longitude
train_data = train_data[abs(train_data["Y"])<38]
train_data = train_data.drop(['Resolution', 'Descript'], axis=1)
train_data['Dates'] = pd.to_datetime(train_data['Dates'])
train_data['Year'], train_data['Month']  = train_data['Dates'].dt.year, train_data['Dates'].dt.month 
train_data['Hour'] = train_data['Dates'].dt.hour
train_data['Latitude'] = train_data['Y']
train_data['Longitude'] = train_data['X']
train_data = train_data.drop(['X', 'Y'], axis=1)
train_data['GeoCode'] = train_data.apply(geoencode, axis=1)
train_data.to_csv("sfcrimedata.csv")   
train_labels = train_data['Category']

In [36]:
#TO DO: remove outlier or erroneous data - lat and long
#TO DO: shuffle the data up





#print('data shape: ', X.shape)
#print('label shape:', Y.shape)

#train_data.head(10)
baseline_train_data_count = 50000
btrain_data = train_data[:(baseline_train_data_count*2)]
btrain_datacopy = btrain_data
btrain_labels = btrain_data['Category']

geocode = pd.get_dummies(btrain_data.GeoCode)
btrain_data = pd.concat([geocode], axis=1)
#btrain_data['Year'] = btrain_datacopy['Year']
btrain_data['Month'] = btrain_datacopy['Month']
btrain_data['Hour'] = btrain_datacopy['Hour']

bdev_data = btrain_data[(baseline_train_data_count+1):]
bdev_labels = btrain_labels[(baseline_train_data_count+1):]
btrain_data = btrain_data[:baseline_train_data_count]
btrain_labels = btrain_labels[:baseline_train_data_count]

#btrain_data.head(10)
#bdev_data.head(10)

#btrain_data = btrain_data.drop(['Category', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'GeoCode', 'Latitude','Longitude'], axis=1)
#btrain_data.head(10)
#bdev_data = train_data[(baseline_train_data_count+1):(baseline_train_data_count*2)]
#bdev_datacopy = bdev_data
#bdev_labels['Category'] = bdev_data['Category']

#geocode = pd.get_dummies(bdev_data.GeoCode)
#bdev_data = pd.concat([geocode], axis=1)
#bdev_data['Month'] = bdev_datacopy['Month']
#bdev_data['Hour'] = bdev_datacopy['Hour']
#bdev_data.head(10)
#bdev_data = bdev_data.drop(['Category', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'GeoCode', 'Latitude','Longitude'], axis=1)
#bdev_data.head(10)

#btest_data = train_data[(baseline_train_data_count*2)+1:]
#btrain_data.head(10)


#train_data['Dates'] = pd.to_datetime(train_data['Dates'])

,9q8ys3x,9q8ys6w,9q8ys7x,9q8ys9e,9q8ys9s,9q8ysb8,9q8yscb,9q8yscu,9q8ysf3,9q8ysfd,...,9q8znd7,9q8znd8,9q8znd9,9q8zndb,9q8zndd,9q8zndh,9q8zndj,9q8zndk,9q8zpe0,9q8zpkc
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# TO DO: Plot a frequency histogram to understand correlation between crime count and each of the following crime type, 
# day of the week, police district, encoded geo-location, year, month and hour
# 

gb = btrain_data.groupby('Category').count()
#type(gb)
#gb
#width = 35
#plt.bar(gb[0], gb[1], width=width, color="blue")
#plt.grid(True)
#plt.xlabel('Vocabulary Size')
#plt.ylabel('F1 Score')
#plt.title('Accuracy (F1-Score) for different Vocabulary Sizes')


In [39]:
# Assign a baseline label to dev dataset and check accuracy
list1 = [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
baselinepredict = np.array([list1], float)
for i in range (2, baseline_train_data_count):
    baselinepredict = np.append(baselinepredict, [list1], axis=0)

#for i in range (2, len(train_data)):
#    baselinepredict = np.append(baselinepredict, [list1], axis=0)

#print(baselinepredict.shape)
print("log loss ",log_loss(bdev_labels, baselinepredict))


# random froest 
#Num_Trees = 400
#rf_model = RandomForestClassifier(n_estimators=Num_Trees, max_depth=50, n_jobs=4)
#rf_model.fit(btrain_data, btrain_labels)
#predicted = np.array(rf_model.predict_proba(bdev_data))
#print('predicted', predicted)
#print( 'Accuracy for Random Forest model with %d trees is: %0.3f' %(Num_Trees, rf_model.score(bdev_data, bdev_labels)))
#log_loss(bdev_labels, predicted)
#print(rf_model.classes_)

#clf = GaussianNB()
#clf.fit(btrain_data, btrain_labels)
#prediction = clf.predict(bdev_data)
#predicted = np.array(clf.predict(bdev_data))
#accuracy=clf.score(bdev_data, bdev_labels)
#print('Accuracy for GaussianNB is: %0.3f' %(accuracy))


log loss  33.8694015209
